In [ ]:
from mdgen.parsing import parse_train_args

from mdgen.logger import get_logger
logger = get_logger(__name__)

import torch, os
from mdgen.dataset import EquivariantTransformerDataset_CrCoNi



torch.set_float32_matmul_precision('medium')
# args = parse_train_args()
    
trainset = EquivariantTransformerDataset_CrCoNi("data/alchem_CrCoNi_data", 2.5, num_frames=3, stage="train")

In [ ]:
'''
sample_traj = []
for i in range(5):
    sample_traj.append(trainset[i])

for i in range(5):
    print(sample_traj[i]["species"].shape)
'''


In [ ]:
'''
# call at stage: save 
num_data = len(trainset)
for i in range(num_data):
    len_traj = trainset[i]
    print(i, num_data, len_traj)
    # print(sample_traj.keys())
    # print(sample_traj["x"].shape)
    # print((sample_traj["cell"]).shape)

'''

In [ ]:
device = "cuda"

In [ ]:
batch_size = 4
train_loader = torch.utils.data.DataLoader(
    trainset,
    batch_size=batch_size,
    num_workers=1,
    shuffle=True,
)
sample_batch = next(iter(train_loader))
sample_batch = {key: value.to(device) if isinstance(value, torch.Tensor) else value 
                for key, value in sample_batch.items()}

In [ ]:
from mdgen.parsing import parse_train_args

from mdgen.logger import get_logger
logger = get_logger(__name__)

import torch, os, wandb
from mdgen.dataset import MDGenDataset
from mdgen.wrapper import NewMDGenWrapper
from pytorch_lightning.callbacks import ModelCheckpoint, ModelSummary
import pytorch_lightning as pl


torch.set_float32_matmul_precision('medium')
# args = parse_train_args()

from argparse import Namespace

args = Namespace(ckpt=None, validate=False, num_workers=4, epochs=10000, overfit=False, overfit_peptide=None, overfit_frame=False, train_batches=None, val_batches=None, val_repeat=25, inference_batches=0, val_freq=None, val_epoch_freq=1, no_validate=False, designability_freq=1, print_freq=100, ckpt_freq=40, wandb=False, run_name='default', accumulate_grad=1, grad_clip=1.0, check_grad=False, grad_checkpointing=False, adamW=False, ema=False, ema_decay=0.999, lr=0.0001, precision='32-true', train_split='splits/4AA_train.csv', val_split='splits/4AA_val.csv', data_dir='data/4AA_data/', num_frames=100, crop=4, suffix="", atlas=False, copy_frames=False, no_pad=False, short_md=False, design_key_frames=False, no_aa_emb=False, no_torsion=False, no_design_torsion=False, supervise_no_torsions=False, supervise_all_torsions=False, no_offsets=False, no_frames=False, hyena=False, no_rope=False, dropout=0.0, scale_factor=1.0, interleave_ipa=False, prepend_ipa=True, oracle=False, num_layers=5, embed_dim=48, mha_heads=16, ipa_heads=4, ipa_head_dim=32, ipa_qk=8, ipa_v=8, time_multiplier=100.0, abs_pos_emb=True, abs_time_emb=False, prediction='velocity', sampling_method='dopri5', alpha_max=8, discrete_loss_weight=0.5, dirichlet_flow_temp=1.0, allow_nan_cfactor=False, tps_condition=True, design_from_traj=False, sim_condition=False, inpainting=False, dynamic_mpnn=False, mpnn=False, frame_interval=None, cond_interval=None)

args.sim_condition=True
args.data_dir="data/alchem_CrCoNi_data" 
args.num_frames=3
args.prepend_ipa=True 
args.abs_pos_emb=True 
args.crop=4 
args.ckpt_freq = 40 
args.val_repeat = 25 
args.epochs = 10
args.num_species = 20
args.edge_dim = 8
args.num_convs = 5
args.num_heads = 4
args.ff_dim = 8
args.cutoff=2.5
args.design=True
args.path_type="Linear"
args.batch_size = batch_size

os.environ["MODEL_DIR"] = os.path.join("workdir", args.run_name)

In [ ]:
from mdgen.equivariant_wrapper import EquivariantMDGenWrapper
model = EquivariantMDGenWrapper(args).to(device)

In [ ]:
import time

model.iter_step += 1
model.stage = "train"
start1 = time.time()
prep = model.prep_batch(sample_batch)
start = time.time()
print("prep time:", start-start1)

In [ ]:
print(prep.keys())
print(prep["model_kwargs"].keys())

In [ ]:
from mdgen.model.utils.data_utils import (
    frac_to_cart_coords_with_lattice,
    get_pbc_distances,
    lattice_params_to_matrix_torch,
    radius_graph_pbc,
)
from mdgen.model.gemnet.utils import (
    repeat_blocks,
)
otf_graph = True

In [ ]:
print(prep['model_kwargs']['num_atoms'][0])
print(prep['latents'].shape)
print(prep['loss_mask'].shape)
print(prep['model_kwargs']['cell'].shape)


## Test transport algo

In [ ]:
x1 = prep["latents"]
aatype1=prep["species"]
mask=prep["loss_mask"]
model_kwargs=prep["model_kwargs"]
transport = model.transport

In [ ]:
t, x0, x1 = transport.sample(x1)
# t, xt, ut = transport.path_sampler.plan(t, x0, x1)
print(t.shape)

In [ ]:
# model_output = model.model(xt,t,**model_kwargs)

In [ ]:
from mdgen.transport.transport import t_to_alpha
seq_one_hot = aatype1
alphas, _ = t_to_alpha(t, transport.args)

In [ ]:
alphas = torch.ones_like(seq_one_hot) + seq_one_hot * (alphas[:, None, None, None] - torch.ones_like(seq_one_hot))
x_d = torch.distributions.Dirichlet(alphas).sample()
xt = x_d

In [ ]:
# print(alphas[0][0][:5]) # Batch = 0, Timestep = 0, Atoms = 0~5
# print(xt[0][0][:5]) # Batch = 0, Timestep = 0, Atoms = 0~5
print(xt.shape)
print(alphas.shape)
print(t.shape)
print(xt.device)

## test equivariant_latent_model

In [ ]:
model_output = model.model(xt, t, **model_kwargs)

In [ ]:

print(len(model_output))
print(model_output.shape)


In [ ]:

out_dict = model.transport.training_losses(
    model=model.model,
    x1=prep['latents'],
    aatype1=prep['species'],
    mask=prep['loss_mask'],
    model_kwargs=prep['model_kwargs']
)
print(out_dict)

In [ ]:
print(aatype1.argmax(dim=-1).shape)
print(out_dict["logits"].shape)
print(out_dict["logits"])

In [ ]:
# log_probs = torch.nn.functional.log_softmax(out_dict["logits"], dim=-1)
print(torch.nn.functional.cross_entropy(out_dict["logits"].reshape(-1,5), aatype1.argmax(dim=-1).reshape(-1)))

In [ ]:
print(out_dict.keys())

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.model.parameters(), lr=0.0005)

In [ ]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    Step = 0
    for batch in train_loader:
        # Forward pass
        batch_ = {key: value.to(device) if isinstance(value, torch.Tensor) else value 
                for key, value in batch.items()}
        prep = model.prep_batch(batch_)
        out_dict = model.transport.training_losses(
            model=model.model,
            x1=prep['latents'],
            aatype1=prep['species'],
            mask=prep['loss_mask'],
            model_kwargs=prep['model_kwargs']
        )
        loss = out_dict["loss"].mean()
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        Step += 1
        # print(f'Epoch [{epoch+1}/{num_epochs}], Step [{Step}], Loss: {loss.item():.4f}')
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')